In this assignment we implement a simple version of generating MNIST with the help of HuggingFace's Diffusers library. We will use the MNIST dataset to train a simple model and then use the trained model to generate new images. We will also generate images from a random noise vector.

Hint: You are free to change parts of the script even without a TODO if that helps you. In case your computer is too slow, you can also use Google Colab to run this notebook.

In [ ]:
# install the hugging face diffusers library
!pip install diffusers

Now we create a training configuration for the model.

In [ ]:
from dataclasses import dataclass
from tqdm.auto import tqdm
from PIL import Image
import torch
import torchvision

# TODO adapt these parameters such that they work for your setup
@dataclass
class TrainingConfig:
    image_size = 28  # the generated image resolution
    num_channels = 1  # the number of channels in the generated image
    train_batch_size = 5
    eval_batch_size = 2  # how many images to sample during evaluation
    num_epochs = 10
    learning_rate = 1e-4
    output_dir = "samples"

config = TrainingConfig()

num_train_timesteps=1000
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Next we will create the MNIST dataset and the dataloader from it

In [ ]:
import torchvision.transforms as transforms
mnist_dataset = torchvision.datasets.MNIST(root='mnist_data', train=True, download=True, transform=transforms.ToTensor())
train_dataloader = torch.utils.data.DataLoader(mnist_dataset, batch_size=config.train_batch_size, shuffle=True, num_workers=1)

Create a variance schedule for calculating the alphas and alpha_overline based on https://arxiv.org/pdf/2006.11239.pdf - we use a simple linear schedule here, but quadratic or sinusoidal schedules are also possible:

In [ ]:
def get_alpha(timesteps, num_timesteps, train_timesteps = None):
    if not train_timesteps is None:
        timesteps = (timesteps * train_timesteps / num_timesteps).to(torch.int32)
        num_timesteps = train_timesteps

    betas = torch.linspace(0.0001, 0.02, num_timesteps).to(timesteps.device)
    return torch.tensor([1 - betas[timesteps[t]] for t in range(len(timesteps))]).to(timesteps.device)

def get_alpha_overline(timesteps, num_timesteps, train_timesteps = None):
    if not train_timesteps is None:
        timesteps = (timesteps * train_timesteps / num_timesteps).to(torch.int32)
        num_timesteps = train_timesteps

    betas = torch.linspace(0.0001, 0.02, num_timesteps).to(timesteps.device)
    return torch.tensor([torch.prod(1 - betas[:timesteps[t]]) for t in range(len(timesteps))]).to(timesteps.device)

Next you will implement the forward diffusion process:

In [ ]:
def forward_diffusion(clean_images, noise, timesteps, num_timesteps):
    # TODO implement the forward diffusion process - 10 points
    # it takes the clean images, the noise and the timesteps as input and returns the noisy images
    alpha = get_alpha_overline(timesteps, num_train_timesteps)[:, None, None, None].to(clean_images.device)
    return torch.sqrt(alpha) * clean_images + torch.sqrt(1 - alpha) * noise

Sanity check forward diffusion process:

In [ ]:
def save_and_show(batch, name, nrow=1):
    image_grid = torchvision.utils.make_grid(batch, nrow)
    torchvision.utils.save_image(image_grid, name)
    Image.open(name)

sample_batch = next(iter(train_dataloader))[0]
noise = torch.randn_like(sample_batch)
save_and_show(sample_batch, 'original.png')
for i in range(num_train_timesteps):
    if i % 100 == 0:
        current_batch = forward_diffusion(sample_batch, noise, i * torch.ones([sample_batch.shape[0]], dtype=torch.int32), num_train_timesteps)
        save_and_show(current_batch, f'forward_diffusion_{i}.png')


Moreover, we create the actual U-Net model:

In [ ]:
from diffusers import UNet2DModel

model = UNet2DModel(
    sample_size=config.image_size,  # the target image resolution
    in_channels=1,  # the number of input channels, 3 for RGB images
    out_channels=1,  # the number of output channels
    layers_per_block=2,  # how many ResNet layers to use per UNet block
    block_out_channels=(64, 128, 128),  # the number of output channels for each UNet block
    down_block_types=(
        "DownBlock2D",  # a regular ResNet downsampling block
        "DownBlock2D",  # a ResNet downsampling block with spatial self-attention
        "DownBlock2D",  # a regular ResNet downsampling block
    ),
    up_block_types=(
        "UpBlock2D",  # a regular ResNet upsampling block
        "UpBlock2D",  # a ResNet upsampling block with spatial self-attention
        "UpBlock2D",  # a regular ResNet upsampling block
    ),
)

Now we create the main train loop:

In [ ]:
import os
import torch.nn.functional as F
import torch
from PIL import Image

optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)

def train_loop(config, model, forward_diffusion, optimizer, train_dataloader, device, num_train_timesteps):
    model.to(device)
    global_step = 0
    sample_batch = next(iter(train_dataloader))[0].to(device)
    save_and_show(sample_batch, f'reconstruction_original.png')
    test_noise = torch.randn_like(sample_batch)

    # Now you train the model
    for epoch in range(config.num_epochs):
        progress_bar = tqdm(total=len(train_dataloader))
        progress_bar.set_description(f"Epoch {epoch}")

        for step, batch in enumerate(train_dataloader):
            if epoch * len(train_dataloader) + step in [0, 100, 200, 1000, 5000, 10000, 20000]:
                noisy_images_list = []
                sample_batch_reconstructed = []
                for noise_level in range(11):
                    timesteps = int(num_train_timesteps * noise_level / 10) * torch.ones([sample_batch.shape[0]], dtype=torch.int32)
                    #timesteps = torch.tensor([0]).to(device)
                    # alpha = 1 - (timesteps / num_train_timesteps)[:, None, None, None].to(device)
                    alpha = get_alpha_overline(timesteps, num_train_timesteps)[:, None, None, None].to(device)
                    # noisy_images = torch.sqrt(alpha) * sample_batch + torch.sqrt(1 - alpha) * noise
                    noisy_images = forward_diffusion(sample_batch, test_noise, timesteps, num_train_timesteps)
                    noisy_images_list.append(noisy_images)
                    noise_pred = model(noisy_images, timesteps.to(device), return_dict=False)[0].detach()
                    sample_batch_reconstructed.append((noisy_images - torch.sqrt(1 - alpha) * noise_pred) / torch.sqrt(alpha))
                    
                save_and_show(torch.cat(noisy_images_list, 0), f'noisy_images_{epoch}_{step}.png', nrow=sample_batch.shape[0])
                save_and_show(torch.cat(sample_batch_reconstructed, 0), f'reconstruction_{epoch}_{step}.png', nrow=sample_batch.shape[0])

            clean_images = batch[0].to(device)
            # Sample noise to add to the images
            noise = torch.randn(clean_images.shape).to(device)
            bs = clean_images.shape[0]

            # Sample a random timestep for each image
            timesteps = torch.randint(
                0, num_train_timesteps, (bs,), device=clean_images.device
            ).long().to(device)

            # Add noise to the clean images according to the noise magnitude at each timestep
            # (this is the forward diffusion process)
            noisy_images = forward_diffusion(clean_images, noise, timesteps, num_train_timesteps)

            # Predict the noise residual
            noise_pred = model(noisy_images, timesteps, return_dict=False)[0]
            loss = F.mse_loss(noise_pred, noise)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            progress_bar.update(1)
            logs = {"loss": loss.detach().item(), "step": global_step}
            progress_bar.set_postfix(**logs)
            global_step += 1

Now we can train the model:

In [ ]:
train_loop(config, model, forward_diffusion, optimizer, train_dataloader, device, num_train_timesteps)

In [ ]:
torch.save(model, 'unet.pt')

In [ ]:
model = torch.load('unet_old.pt').to(device)

Now we create the utils for evaluating the model:

In [ ]:
def evaluate(config, img_name, reverse_diffusion_process, model, timesteps, device, train_timesteps):
    #
    noise = torch.randn([config.eval_batch_size, config.num_channels, config.image_size, config.image_size])
    images = reverse_diffusion_process(model, noise, timesteps, device, train_timesteps)
    # Make a grid out of the images
    image_grid = torchvision.utils.make_grid(images) #make_grid(images, rows=4, cols=4)

    # Save the image grid
    torchvision.utils.save_image(image_grid, img_name)

Now you will implement the reverse diffusion process with DDPM:

In [ ]:
def reverse_diffusion_ddpm(model, noise, num_timesteps, device, train_timesteps):
    # TODO implement the reverse diffusion process with DDPM - 15 points
    # it should take noise, the model and the number of timesteps as input and return the generated images
    # Generate the initial image from the noise
    noisy_images_list = []
    current_image = noise.to(device)
    
    # Perform reverse diffusion for the specified number of timesteps
    progress_bar = tqdm(range(num_timesteps - 1))
    #for t in range(num_timesteps - 1, -1, -1):
    for t in range(1, num_timesteps - 1)[::-1]:
        # Generate the noise for the current timestep
        progress_bar.set_description(f"T {t}")
        timesteps = torch.ones([current_image.shape[0]], dtype=torch.int32).to(device) * t
        current_noise = model(current_image, timesteps, return_dict=False)[0]

        #
        alpha = get_alpha(timesteps, num_timesteps, train_timesteps)[:,None,None,None]
        alpha_overline = get_alpha_overline(timesteps, num_timesteps, train_timesteps)[:,None,None,None]
        # sigma = torch.tensor(0).to(device)
        sigma = torch.tensor(0.01 * (t / num_timesteps)) if t > 1 else torch.tensor(0).to(device)
        z = torch.randn_like(current_image).to(device)

        #
        current_image = 1 / alpha * (current_image - (1 - alpha) / torch.sqrt(1 - alpha_overline) * current_noise) + sigma * z
        current_image = current_image.detach()
        if t % (num_timesteps / 10) == 0:
            noisy_images_list.append(current_image)
    
    save_and_show(torch.cat(noisy_images_list, 0), f'ddpm_collage.png', nrow=current_image.shape[0])
    # Return the generated images
    return current_image


And sample from it:

In [ ]:
#num_inference_timesteps = num_train_timesteps
num_inference_timesteps = 200
evaluate(config, 'ddpm.png', reverse_diffusion_ddpm, model, num_inference_timesteps, device, num_train_timesteps)
Image.open('ddpm.png')

# Here the generated MNIST digits should be printed if you implemented it correctly!

The outputs should look like this:

In [ ]:
image_grid = torchvision.utils.make_grid(next(iter(train_dataloader))[0])
torchvision.utils.save_image(image_grid, 'example.png')
Image.open('example.png')

Now you will implement the reverse diffusion process with DDIM:

In [ ]:
def reverse_diffusion_ddim(model, noise, num_timesteps, device, train_timesteps):# TODO implement the reverse diffusion process with DDPM - 15 points
    # it should take noise, the model and the number of timesteps as input and return the generated images
    # Generate the initial image from the noise
    noisy_images_list = []
    current_image = noise.to(device)
    
    # Perform reverse diffusion for the specified number of timesteps
    progress_bar = tqdm(range(num_timesteps - 1))
    #for t in range(num_timesteps - 1, -1, -1):
    for t in range(1, num_timesteps)[::-1]:
        # Generate the noise for the current timestep
        progress_bar.set_description(f"T {t}")
        timesteps = torch.ones([current_image.shape[0]], dtype=torch.int32).to(device) * t
        current_noise = model(current_image, timesteps, return_dict=False)[0]

        #
        #alpha_previous = get_alpha(timesteps - 1, num_timesteps, train_timesteps)[:,None,None,None]
        #alpha = get_alpha(timesteps, num_timesteps, train_timesteps)[:,None,None,None]
        alpha_previous = get_alpha_overline(timesteps - 1, num_timesteps, train_timesteps)[:,None,None,None]
        alpha = get_alpha_overline(timesteps, num_timesteps, train_timesteps)[:,None,None,None]
        sigma = torch.tensor(0).to(device)
        #sigma = torch.tensor(0.01 * (t / num_timesteps)) if t > 1 else torch.tensor(0).to(device)
        z = torch.randn_like(current_image).to(device)

        #
        next_image = torch.sqrt(alpha_previous) * (current_image - torch.sqrt(1 - alpha) * current_noise) / torch.sqrt(alpha)
        next_image += torch.sqrt(1 - alpha_previous - torch.square(sigma)) * current_noise
        next_image += sigma * z
        current_image = next_image
        current_image = current_image.detach()
        if t % (num_timesteps / 10) == 0:
            noisy_images_list.append(current_image)
    
    save_and_show(torch.cat(noisy_images_list, 0), f'ddim_collage.png', nrow=current_image.shape[0])
    # Return the generated images
    return current_image

And evaluate with it:

In [ ]:
#num_inference_timesteps = num_train_timesteps
num_inference_timesteps = 200
evaluate(config, 'ddim.png', reverse_diffusion_ddim, model, num_inference_timesteps, device, num_train_timesteps)
Image.open('ddim.png')

# Here the generated MNIST digits should be printed if you implemented it correctly!

In [ ]:
# Bonus exercise:
# Compare different num_inference_timesteps and different models (DDPM vs DDIM) and discuss the results.
